In [20]:
# 1. 导入所有依赖库（新增numpy，用于固定随机种子）
import pandas as pd
import jieba.posseg as pseg
from gensim import corpora
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Gamma
import numpy as np  # 新增：用于固定numpy随机种子


# 2. 基础配置与GPU验证（先确认GPU可用）+ 固定随机种子（核心新增）
def set_random_seed(seed=2023):
    """固定所有随机种子，确保结果可重复"""
    # 固定PyTorch随机种子
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # 多GPU时用
    # 固定NumPy随机种子（代码中用numpy处理数据）
    np.random.seed(seed)
    # 禁用PyTorch的cuDNN随机性（确保卷积等操作一致）
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [27]:

# 调用函数固定随机种子（seed=42是常用值，可自行修改）
set_random_seed(seed=2023)

In [29]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"当前运行设备：{device}")  # 确保输出"cuda"
print(f"GPU是否可用：{torch.cuda.is_available()}")


# 3. 读取并预处理数据（过滤空值+空文档）
def load_and_preprocess_data(csv_path):
    # 读取CSV
    df = pd.read_csv(csv_path, encoding="gbk")
    # 过滤message为空的行
    df = df.dropna(subset=["message"]).reset_index(drop=True)
    messages = df["message"].tolist()
    
    # 停用词（内置，无需额外文件）
    stopwords = {
        "我", "你", "他", "她", "它", "我们", "你们", "他们", "的", "了", "是", "在", 
        "有", "和", "及", "与", "也", "还", "都", "很", "挺", "太", "非常", "呵呵", 
        "哈哈", "嗯", "哦", "呀", "啊", "啦", "吧", "呢", "吗", "今天", "昨天", "明天",
        "。", "，", "、", "；", "：", "？", "！", "（", "）", "【", "】", "《", "》"
    }
    keep_flags = ['n', 'v', 'a', 'vn']  # 保留名词、动词、形容词
    
    # 分词+过滤（关键：过滤空文档）
    processed_words = []
    valid_indices = []  # 记录有效文档的索引（用于后续筛选df）
    for idx, msg in enumerate(messages):
        # 清洗特殊字符（保留中文、字母、数字）
        msg_clean = "".join([c for c in str(msg) if c.isalnum() or '\u4e00' <= c <= '\u9fff'])
        # 分词+词性过滤
        words = [w for w, f in pseg.cut(msg_clean) if f in keep_flags and w not in stopwords and len(w) > 1]
        if words:  # 只保留有有效词的文档
            processed_words.append(words)
            valid_indices.append(idx)
    
    # 筛选有效文档的df（避免后续索引不匹配）
    df_valid = df.iloc[valid_indices].reset_index(drop=True)
    print(f"原始数据量：{len(messages)}条 → 过滤后有效数据量：{len(processed_words)}条")
    return df_valid, processed_words


# 4. 构建词典与GPU张量（适配PyTorch）
def build_corpus_and_tensor(processed_words, device):
    # 构建gensim词典（过滤低频词）
    dictionary = corpora.Dictionary(processed_words)
    dictionary.filter_extremes(no_below=2)  # 保留至少在2条文档中出现的词
    n_words = len(dictionary)
    n_docs = len(processed_words)
    
    # 词袋向量转PyTorch GPU张量（避免稀疏矩阵问题）
    doc_word_tensor = torch.zeros((n_docs, n_words), dtype=torch.float32, device=device)
    for doc_idx, bow in enumerate([dictionary.doc2bow(words) for words in processed_words]):
        for word_idx, count in bow:
            doc_word_tensor[doc_idx, word_idx] = count
    
    return dictionary, doc_word_tensor, n_words, n_docs


# 5. 稳定版GPU-LDA模型（Gamma初始化+防nan）
class StableLDA(nn.Module):
    # 修复：新增n_docs参数（避免之前的NameError）
    def __init__(self, n_topics, n_words, n_docs, alpha=1.0, beta=0.1, device=device):
        super().__init__()
        self.n_topics = n_topics
        self.n_words = n_words
        self.n_docs = n_docs  # 存储文档数
        
        # 关键：Gamma分布初始化（因已固定随机种子，每次初始化参数完全一致）
        self.log_topic_word = nn.Parameter(
            torch.log(
                Gamma(concentration=beta*10, rate=10).sample((n_topics, n_words)).to(device)
            )
        )
        self.log_doc_topic = nn.Parameter(
            torch.log(
                Gamma(concentration=alpha*10, rate=10).sample((n_docs, n_topics)).to(device)
            )
        )

    def forward(self):
        # 计算概率分布（softmax确保和为1）
        doc_topic = torch.softmax(self.log_doc_topic, dim=1)  # 文档→主题
        topic_word = torch.softmax(self.log_topic_word, dim=1)  # 主题→词
        doc_word_pred = torch.matmul(doc_topic, topic_word)  # 文档→词预测
        return doc_word_pred, doc_topic, topic_word


# 6. 训练与结果提取（防nan损失+主题差异化）
def train_lda_and_extract_results(
    doc_word_tensor, n_topics, n_words, n_docs, df_valid, dictionary,
    lr=0.001, n_epochs=1000, device=device
):
    # 初始化模型、优化器、损失函数（因固定种子，每次初始化完全一致）
    model = StableLDA(
        n_topics=n_topics, 
        n_words=n_words, 
        n_docs=n_docs,  # 传入文档数
        device=device
    ).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.KLDivLoss(reduction="sum")  # LDA专用损失
    
    # 训练循环（因固定种子，每次训练的损失变化和参数更新完全一致）
    print("\n开始训练LDA模型...")
    for epoch in range(n_epochs):
        model.train()
        optimizer.zero_grad()
        
        # 前向传播
        doc_word_pred, doc_topic, topic_word = model()
        # 计算稳定损失（加1e-20避免log(0)和除以0）
        doc_word_norm = doc_word_tensor / (doc_word_tensor.sum(dim=1, keepdim=True) + 1e-20)
        loss = loss_fn(torch.log(doc_word_pred + 1e-20), doc_word_norm)
        
        # 反向传播+更新
        loss.backward()
        optimizer.step()
        
        # 每200次迭代打印损失（验证无nan，且每次运行损失值完全相同）
        if (epoch + 1) % 200 == 0:
            print(f"迭代 {epoch+1}/{n_epochs} | 损失：{loss.item():.2f}")
    
    # 提取结果（因固定种子，主题关键词和文档分配完全一致）
    with torch.no_grad():
        _, doc_topic, topic_word = model()
        # 文档→主题结果（转CPU处理）
        doc_topic_cpu = doc_topic.cpu().numpy()
        df_valid["topic_id"] = [doc_topic_cpu[i].argmax() for i in range(n_docs)]  # 主主题ID
        df_valid["topic_prob"] = [doc_topic_cpu[i].max() for i in range(n_docs)]    # 主主题概率
        
        # 主题→关键词结果（取前10个词）
        topic_word_cpu = topic_word.cpu().numpy()
        topic_keywords = []
        for topic_idx in range(n_topics):
            top_word_ids = topic_word_cpu[topic_idx].argsort()[-10:][::-1]  # 权重最高的10个词ID
            top_words = [dictionary[id] for id in top_word_ids]
            topic_keywords.append(f"主题{topic_idx}：{', '.join(top_words)}")
    
    return df_valid, topic_keywords


# 7. 主函数（一键运行全流程）
if __name__ == "__main__":
    # ---------------------- 配置参数（只需改这里） ----------------------
    CSV_PATH = "南京景区-天气-社媒情感融合表.csv"  # 替换为你的原始CSV路径
    N_TOPICS = 4  # 主题数
    SAVE_PATH = "全量数据_带主题_最终版.csv"  # 结果保存路径
    
    # ---------------------- 执行全流程 ----------------------
    # 1. 数据预处理（确定性操作，无随机）
    df_valid, processed_words = load_and_preprocess_data(CSV_PATH)
    # 2. 构建词典与GPU张量（确定性操作，无随机）
    dictionary, doc_word_tensor, n_words, n_docs = build_corpus_and_tensor(processed_words, device)
    # 3. 训练模型并提取结果（因固定种子，结果完全可重复）
    df_result, topic_keywords = train_lda_and_extract_results(
        doc_word_tensor=doc_word_tensor,
        n_topics=N_TOPICS,
        n_words=n_words,
        n_docs=n_docs,
        df_valid=df_valid,
        dictionary=dictionary
    )
    # 4. 保存结果并打印主题
    df_result.to_csv(SAVE_PATH, index=False, encoding="utf-8")
    print(f"\n主题关键词列表：")
    for kw in topic_keywords:
        print(kw)
    print(f"\n结果已保存到：{SAVE_PATH}")

当前运行设备：cuda
GPU是否可用：True
原始数据量：5519条 → 过滤后有效数据量：4365条

开始训练LDA模型...
迭代 200/1000 | 损失：26679.63
迭代 400/1000 | 损失：25596.54
迭代 600/1000 | 损失：24599.92
迭代 800/1000 | 损失：23693.67
迭代 1000/1000 | 损失：22890.96

主题关键词列表：
主题0：没有, 世界, 开心, 民国, 感受, 震撼, 出来, 生活, 灵谷, 演唱会
主题1：落羽杉, 开始, 动物园, 游客, 美好, 音乐, 最美, 遇见, 游园, 适合
主题2：故事, 漂亮, 酒家, 愉快, 举手, 关注, 日子, 照片, 美龄, 结束
主题3：日落, 打卡, 总统府, 好看, 自由, 分享, 风景区, 图片, 苦涩, 大道

结果已保存到：全量数据_带主题_最终版.csv
